In [233]:
import numpy as np
from scipy.optimize import minimize
from math import sqrt
import pickle

<h2>Given, nothing to do</h2>

In [234]:
def initializeWeights(n_in, n_out):
    '''
    initializeWeights return the random weights for Neural Network given the
    number of node in the input layer and output layer

    Input:
    n_in: number of nodes of the input layer
    n_out: number of nodes of the output layer

    Output:
    W: matrix of random initial weights with size (n_out x (n_in + 1))
    '''
    epsilon = sqrt(6) / sqrt(n_in + n_out + 1)
    W = (np.random.rand(n_out, n_in + 1) * 2 * epsilon) - epsilon
    return W

In [235]:
def preprocess(filename,scale=True):
#     print("preprocess")
    '''
     Input:
     filename: pickle file containing the data_size
     scale: scale data to [0,1] (default = True)
     Output:
     train_data: matrix of training set. Each row of train_data contains
       feature vector of a image
     train_label: vector of label corresponding to each image in the training
       set
     test_data: matrix of training set. Each row of test_data contains
       feature vector of a image
     test_label: vector of label corresponding to each image in the testing
       set
    '''
    with open(filename, 'rb') as f:
        train_data = pickle.load(f)
        train_label = pickle.load(f)
        test_data = pickle.load(f)
        test_label = pickle.load(f)
    # convert data to double
    train_data = train_data.astype(float)
    test_data = test_data.astype(float)

    # scale data to [0,1]
    if scale:
        train_data = train_data/255
        test_data = test_data/255

    return train_data, train_label, test_data, test_label

<h2>TO DO</h2>

In [304]:
def sig(z):
    return 1/(1 + np.exp(-z))

def sigmoid(z):
    '''
    Notice that z can be a scalar, a vector or a matrix
    return the sigmoid of input z (same dimensions as z)
    '''
    # using numpy ufunc to vectorize function

    s = sig(z)

    return s

# weighed sums
def applyw(w_j, x):
    # where w_j is the weight corresponding with hidden unit j
    # x is a feature vector
    d = len(x)
    bias = w_j[3] * x[3]
    q = np.multiply(w_j, x)
    sum = np.sum(q)
    sum = sum - bias
    return sum

def applywb(w_j, x):
    # where w_j is the weight corresponding with hidden unit j
    # x is a feature vector
    d = len(x)
    #bias = w_j[0] * x[0]
    q = np.multiply(w_j, x)
    sum = np.sum(q)
    #sum = sum - bias
    return sum

# 1 of K y encoding
def encode(y):
    k = 2
    i = len(y)
    Y = np.zeros((i, k))
    nl = np.arange(i)
    Y[(nl), (y)] = 1
    return Y

def applog(z):
    return (np.log(z))

def logz(z):
    s = applog(z)
    return s

def applogm(z):
    return (np.log(1 - z))

def logm(z):
    s = applogm(z)
    return s

def appsuby(z):
    return (1 - z)

def suby(z):
    s = appsuby(z)
    return s
# log like error function for each input data
def errfuncsig(o, y, n):
    # y_i is the vector of 1 of k encoded y matrix at row i
    logT = logz(o)
    logM = logm(o)
    ym = suby(y)
    m_1 = logT * y
    m_2 = ym * logM
    sum_m = m_1 + m_2
    a = sum_m.sum()
    return (a * (-1/n))
# gradient with respect to weights of error functions
def graderror(y_i, o_i, x_i):
    theta = y_i - o_i
    nl = theta * x_i
    return nl

# errfunsum is the final summation of all of the error vals
def errfunsum(sum_v):
    n = len(sum_v)
    sum = np.sum(sum_v)
    return (1/n) * sum

In [307]:
def nnObjFunction(params, *args):
    '''
    % nnObjFunction computes the value of objective function (cross-entropy
    % with regularization) given the weights and the training data and lambda
    % - regularization hyper-parameter.

    % Input:
    % params: vector of weights of 2 matrices W1 (weights of connections from
    %     input layer to hidden layer) and W2 (weights of connections from
    %     hidden layer to output layer) where all of the weights are contained
    %     in a single vector.
    % n_input: number of node in input layer (not including the bias node)
    % n_hidden: number of node in hidden layer (not including the bias node)
    % n_class: number of node in output layer (number of classes in
    %     classification problem
    % train_data: matrix of training data. Each row of this matrix
    %     represents the feature vector of a particular image
    % train_label: the vector of truth label of training images. Each entry
    %     in the vector represents the truth label of its corresponding image.
    % lambda: regularization hyper-parameter. This value is used for fixing the
    %     overfitting problem.

    % Output:
    % obj_val: a scalar value representing value of error function
    % obj_grad: a SINGLE vector (not a matrix) of gradient value of error function
    % NOTE: how to compute obj_grad
    % Use backpropagation algorithm to compute the gradient of error function
    % for each weights in weight matrices.
    '''
    n_input, n_hidden, n_class, train_data, train_label, lambdaval = args
    # First reshape 'params' vector into 2 matrices of weights W1 and W2

    W1 = params[0:n_hidden * (n_input + 1)].reshape((n_hidden, (n_input + 1)))
    W2 = params[(n_hidden * (n_input + 1)):].reshape((n_class, (n_hidden + 1)))
    obj_val = 0
    data = training_data.shape[0]
    d = training_data.shape[1]
    # train_data.shape[0] = 60,000
    # train_data.shape[1] = 784
    
    print(W1.shape[0])
    print(W1.shape[1])
    print(training_data.shape)
    a = np.zeros((data, n_hidden))
    sum = 0
    for i in range(0,data):
        for j in range(0,n_hidden):
            for p in range(0,d+1):
                if p == d:
                    sum += W1[j][p] * 1       # <--- using this as bias for input
                else:
                    sum += W1[j][p] * training_data[i][p]
                    
            a[i][j] = sum
            sum = 0
    
#     print("a")
#     print(a.shape)
#     print(a)
    z = sigmoid(a)
#     print(z)
    ones = np.ones((z.shape[0],1))
    z = np.append(z,ones,axis=1)           # <--- adds bias to z
#     print("z with bias")
#     print(z.shape)

    
#     print("W2")
#     print(W2.shape)
#     print(W2)
    
    
    # OUTPUT UNITS
    b = np.zeros((data, n_class))
    sum = 0
    for i in range(0,data):
        for l in range(0,n_class):
            for j in range(0,n_hidden+1):
                sum += W2[l][j] * z[i][j]
            b[i][l] = sum
            sum = 0
            
#     print("b")
#     print(b.shape)
#     print(b)
    
    o = sigmoid(b)
    

    # ----- FIND obj_val (using equation (5)) -----
    sum = 0
#     for i in range(0,data):
#         for l in range(0,n_class):
#             sum += (training_label[i] * np.log(o[i][l]) + \
#                     (1 - training_label[i]) * np.log(1 - o[i][l]))
# #             print(sum)
    
    y = encode(training_label)
    
    obj_val = errfuncsig(o, y, data)

    Y = np.zeros((2,2))
    
    
    # ----- FIND obj_grad (using equations (16) & (17)) -----
    
    obj_grad = np.zeros(params.shape)
    
    #grad_W1 = np.shape()
    
    
    

    return (obj_val, obj_grad)

<h2>Sample data to test your nnObjFunction() [from Piazza]</h2>

In [308]:


# Paste your sigmoid function here


# Paste your nnObjFunction here


n_input = 5
n_hidden = 3
n_class = 2
training_data = np.array([np.linspace(0,1,num=5),np.linspace(1,0,num=5)])
training_label = np.array([0,1])
lambdaval = 0
params = np.linspace(-5,5, num=26)
args = (n_input, n_hidden, n_class, training_data, training_label, lambdaval)
objval,objgrad = nnObjFunction(params, *args)
print("Objective value:")
print(objval)
print("Gradient values: ")
print(objgrad)

3
6
(2, 5)
Objective value:
7.8716750659708925
Gradient values: 
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0.]
